In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import csv
import time

# Range of pet IDs
start_id = 2101
end_id = 2200  # inclusive

# Setup Chrome options (headless mode optional)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Setup WebDriver
service = Service()  # Adjust path to chromedriver if needed
driver = webdriver.Chrome(service=service, options=chrome_options)

# Write CSV header
with open('pets_selenium.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow([
        "Pet ID", "Name", "Type", "Species", "Profile", "Amount", "Vaccinated", "Dewormed",
        "Spayed", "Condition", "Body", "Color", "Location", "Posted", "Price",
        "Uploader Type", "Uploader Name", "Status"
    ])

    # Loop over each pet ID
    for pet_id in range(start_id, end_id + 1):
        url = f"https://www.petfinder.my/pets/{pet_id}/"
        try:
            driver.get(url)
            time.sleep(1)  # Let the page load

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Extract pet name
            pet_title_tag = soup.find('div', class_='pet_title')
            pet_name = pet_title_tag.find('td', align="center").text.strip() if pet_title_tag else "N/A"

            # Extract pet details
            info_table = soup.find('table', class_='pet_box')
            pet_details = {}

            if info_table:
                rows = info_table.find_all('tr')
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 2:
                        key_tag = cols[0].find('b')
                        if key_tag:
                            key = key_tag.text.strip().replace(":", "")
                            value = cols[1].text.strip()
                            pet_details[key] = value

            # Type and Species
            pet_type = next(iter(pet_details.keys()), "N/A")
            pet_species = pet_details.get(pet_type, "N/A")

            # Price/Adoption Fee
            adoption_fee = "N/A"
            for row in info_table.find_all('tr') if info_table else []:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    key_tag = cols[0].find('b')
                    if key_tag and 'Adoption Fee' in key_tag.text:
                        fee_tag = cols[1].find('b')
                        adoption_fee = fee_tag.text.strip() if fee_tag else cols[1].text.strip()

            # Uploader
            uploader_td = soup.find('td', align="left", width="130", valign="middle")
            uploader_type = uploader_td.find('font').text.strip() if uploader_td and uploader_td.find('font') else "N/A"
            uploader_name_tag = uploader_td.find('a', class_='darkgrey') if uploader_td else None
            uploader_name = uploader_name_tag.text.strip() if uploader_name_tag else "N/A"

            # Status
            status_tag = soup.find('div', class_='pet_label')
            pet_status = status_tag.text.strip() if status_tag else "N/A"

            # Write to CSV
            csv_writer.writerow([
                pet_id,
                pet_name,
                pet_type,
                pet_species,
                pet_details.get('Profile', 'N/A'),
                pet_details.get('Amount', 'N/A'),
                pet_details.get('Vaccinated', 'N/A'),
                pet_details.get('Dewormed', 'N/A'),
                pet_details.get('Spayed', 'N/A'),
                pet_details.get('Condition', 'N/A'),
                pet_details.get('Body', 'N/A'),
                pet_details.get('Color', 'N/A'),
                pet_details.get('Location', 'N/A'),
                pet_details.get('Posted', 'N/A'),
                adoption_fee,
                uploader_type,
                uploader_name,
                pet_status
            ])
            print(f"Scraped pet ID: {pet_id}")
        except Exception as e:
            print(f"Failed to scrape pet {pet_id}: {e}")

# Close WebDriver
driver.quit()
